<a href="https://colab.research.google.com/github/alihaydarkir/fine-tuning/blob/main/sigorta_alaninda_sorgular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. GEREKLI KÜTÜPHANELERI YÜKLEME

In [16]:
!pip install -q transformers datasets accelerate torch evaluate
!pip install -q sentencepiece  # Tokenizer için

import warnings
warnings.filterwarnings("ignore")

from huggingface_hub import login
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    EarlyStoppingCallback
)
from datasets import load_dataset, Dataset
import torch
import os

print("✅ Kütüphaneler yüklendi!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.7 MB/s eta 0:00:00
✅ Kütüphaneler yüklendi!


# 2. HUGGING FACE'E GİRİŞ YAPMA

# 3. GERÇEK TÜRKÇE MODEL SEÇENEKLERİ

In [ ]:

print("\n🇹🇷 Türkçe model seçenekleri kontrol ediliyor...")

# Gerçek ve test edilmiş Türkçe modeller
TURKISH_MODELS = {
    1: "ytu-ce-cosmos/turkish-gpt2",           # YTU GPT-2 Türkçe
    2: "gorkemgoknar/gpt2-small-turkish",      # Küçük Türkçe GPT-2
    3: "cenkersisman/gpt2-turkish-cased",      # Case-sensitive
    4: "redrussianarmy/gpt2-turkish-cased",    # Alternatif Türkçe
    5: "microsoft/DialoGPT-small",             # Dialog odaklı (İngilizce)
    6: "gpt2"                                  # Fallback - temel GPT-2
}

def load_turkish_model():
    """En uygun Türkçe modeli yükle"""
    for model_id, model_name in TURKISH_MODELS.items():
        try:
            print(f"📦 Test ediliyor: {model_name}")

            # Tokenizer yükle
            tokenizer = AutoTokenizer.from_pretrained(model_name)

            # Model yükle
            model = AutoModelForCausalLM.from_pretrained(
                model_name,
                torch_dtype=torch.float32,  # Uyumluluk için
                device_map="auto" if torch.cuda.is_available() else None
            )

            # Pad token ayarla
            if tokenizer.pad_token is None:
                if tokenizer.eos_token:
                    tokenizer.pad_token = tokenizer.eos_token
                else:
                    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
                    model.resize_token_embeddings(len(tokenizer))

            print(f"✅ Başarılı: {model_name}")
            print(f"📊 Parametreler: ~{model.num_parameters()/1e6:.1f}M")
            return model, tokenizer, model_name

        except Exception as e:
            print(f"❌ Başarısız: {model_name} - {str(e)[:50]}...")
            continue

    raise Exception("❌ Hiçbir model yüklenemedi!")

# Model yükle
model, tokenizer, MODEL_NAME = load_turkish_model()
print(f"🎯 Seçilen model: {MODEL_NAME}")


# 4. GERÇEK TÜRKÇE DATASET YÜKLE

In [ ]:
# Gerekli kütüphaneyi yüklemek için:
# pip install datasets

from datasets import Dataset, load_dataset

print("\n📚 Sigortacılık konulu Türkçe dataset yükleniyor...")

def load_turkish_dataset():
    """Türkçe dataset yükleme - sırayla dene"""

    # Dataset seçenekleri - Sadece manuel olanı aktif bıraktık
    dataset_options = [
        # ("turkish-nlp/turkish-instructions", "train", 1000),
        # ("oscar", "unshuffled_deduplicated_tr", 500),
        # ("cc100", "tr", 800),
        ("manual", None, None)  # Sadece sizin verilerinizle oluşturulacak manuel dataset
    ]

    for ds_name, split_name, sample_size in dataset_options:
        try:
            if ds_name == "manual":
                print("📝 Sizin verilerinizle manuel sigortacılık dataset'i oluşturuluyor...")
                return create_manual_turkish_dataset()

            print(f"📦 {ds_name} yükleniyor...")
            if sample_size:
                dataset = load_dataset(ds_name, split=f"{split_name}[:{sample_size}]")
            else:
                dataset = load_dataset(ds_name, split=split_name)

            if 'text' not in dataset.column_names:
                first_col = dataset.column_names[0]
                dataset = dataset.rename_column(first_col, 'text')

            print(f"✅ {ds_name} yüklendi: {len(dataset)} örnek")
            return dataset

        except Exception as e:
            print(f"⚠️ {ds_name} yüklenemedi: {str(e)[:50]}...")
            continue

    raise Exception("❌ Hiçbir dataset yüklenemedi!")

def create_manual_turkish_dataset():
    """Sizin metinlerinizden oluşturulmuş yüksek kaliteli manuel sigortacılık dataset'i"""

    # Sizin metinlerinizden oluşturulan Soru-Cevap çiftleri
    qa_data = [
        # --- BÖLUM 1: TEMEL TANIMLAR VE KAVRAMLAR ---
        ("E-sigortacılık nedir?", "E-sigortacılık, müşterilerin internet aracılığıyla sigorta poliçesi satın almalarını, sözleşmelerini düzenlemelerini ve ödemelerini elektronik ortamda yapabilmelerini sağlayan bir sistemdir."),
        ("İnternetin küreselleşmedeki rolü nedir?", "İnternet, insanların bilgiye erişimini kolaylaştırması ve elektronik haberleşmeyi artırmasıyla küreselleşmenin önemli unsurlarından biri olarak görülmektedir."),
        ("E-ticaretin sigortacılıktaki yansıması ne olarak adlandırılır?", "E-ticaretin sigortacılığa yansıması e-sigortacılık kavramını doğurmuştur."),
        ("E-sigortacılık sadece poliçe satışı mıdır?", "Hayır, yalnızca bir sigorta poliçesinin çevrimiçi satışı değil, aynı zamanda poliçenin araştırılması ve seçilmesi gibi satın alma sürecindeki birçok aşamayı da içermektedir."),
        ("Sigortanın tanımı nedir?", "Sigorta, bir kişinin ileride karşılaşabileceği zararı gidermek için önceden ödediği prim karşılığında, bu işle uğraşan bir kuruluşla yaptığı iki taraflı bir sözleşmedir."),
        ("Hayat sigortası ne işe yarar?", "Hayat sigortaları, sigortalının yaşlılığında hayatını rahat sürdürme veya ölümü halinde ailesine aynı imkânı sağlayan sigorta türüdür."),
        ("Hayat dışı sigortalar neleri kapsar?", "Hayat dışı sigortalar, genellikle kasko, konut gibi maddi hasarların ve ekonomik kayıpların karşılanmasına yönelik sigorta türüdür."),
        ("E-ticaret nasıl tanımlanmaktadır?", "Dünya Ticaret Örgütü'ne göre e-ticaret, mal ve hizmetlerin üretim, reklam, satış ve dağıtımlarının telekomünikasyon ağları üzerinden yapılmasıdır."),
        ("Geleneksel sigortacılık işlemleri nasıl yürütülür?", "Sigortacılık işlemleri genellikle acenteler ve brokerlar aracılığıyla yürütülmektedir."),
        ("İş dünyası için internetin önemi nedir?", "İnternet, şirketler tarafından iletişim, haberleşme, bilgi iletme, reklam ve pazarlamada etkin bir araç ve güçlü bir dağıtım kanalı olarak kullanılır."),

        # --- BÖLUM 2: E-SİGORTA TÜRLERİ VE MODELLERİ ---
        ("En çok tercih edilen e-sigorta türleri hangileridir?", "En çok tercih edilen e-sigorta türleri Şirketten Şirkete (B2B) ve Şirketten Tüketiciye (B2C) modelleridir."),
        ("B2B e-sigorta modeli nedir?", "B2B (Şirketten Şirkete), sigorta şirketlerinin acenteler gibi aracılar vasıtasıyla çevrimiçi sigortacılık yapmasıdır."),
        ("B2B e-sigorta modelini genellikle kimler kullanır?", "Sigorta şirketleri daha çok B2B uygulamasını kullanmaktadır."),
        ("B2C e-sigorta modeli nedir?", "B2C (Şirketten Tüketiciye), sigorta teklifinin veya poliçelerin müşterilere aracısız bir biçimde internetten çevrimiçi olarak oluşturulmasıdır."),
        ("B2C e-sigorta modelini genellikle kimler tercih eder?", "Bu uygulamayı sigorta şirketlerinden çok sigorta aracıları tercih etmektedir."),
        ("C2B e-sigorta ne anlama gelir?", "Tüketiciden şirkete (Consumer to Business) anlamına gelen bir e-sigorta türüdür."),
        ("C2C e-sigorta ne demektir?", "Tüketiciden tüketiciye (Consumer to Consumer) anlamına gelen bir e-sigorta türüdür."),
        ("Mobil ticaretin e-sigortadaki karşılığı nedir?", "Mobil ticaret, m-commerce olarak da bilinen bir e-sigorta türüdür."),
        ("B2E e-sigorta modelinin açılımı nedir?", "Şirketten çalışanlara (Business to Employees) modelidir."),
        ("B2G ve G2C modelleri ne anlama gelir?", "B2G Şirketten Devlete, G2C ise Devletten Tüketiciye anlamına gelen e-sigorta türleridir."),
        ("E-sigorta ürünlerinin tanıtımında hangi araçlar kullanılır?", "Televizyon, telefon, bilgisayar, internet, e-posta ve GSM gibi araçlar tanıtım ve pazarlamada kullanılmaktadır."),

        # --- BÖLUM 3: AVANTAJLAR VE FAYDALAR (MÜŞTERİ) ---
        ("E-sigortanın müşteriler için en önemli avantajı nedir?", "Müşterilerin kendilerine en uygun ürünleri kısa zamanda araştırmalarına olanak sağlamasıdır."),
        ("Müşteriler e-sigorta ile hangi işlemleri yapabilir?", "İnternet üzerinden sigorta poliçesi satın alabilir, sözleşmelerini düzenleyebilir ve ödemelerini yapabilirler."),
        ("E-sigorta, müşterilerin bilgiye erişimini nasıl etkiler?", "Müşteriler, poliçelerle ilgili aracılara bağlı olmadan detaylı bilgi edinebilir ve bu bilgiyi kendi yararlarına kullanabilir ve bu bilgiyi kendi yararlarına kullanabilirler."),
        ("E-sigortacılık prim ödemelerini düşürür mü?", "Evet, genel maliyetler azaldığından sigortalının prim ödemeleri daha düşük olmaktadır."),
        ("Müşteriler e-sigorta sayesinde kişiye özel hizmet alabilir mi?", "Evet, sigortalı kişisel hizmet alabilme imkânı bulmaktadır."),
        ("E-sigorta, müşteriye zaman kazandırır mı?", "Evet, müşteriler sigorta şirketlerine giderek uzun zaman harcamak zorunda kalmazlar."),
        ("Müşteriler e-sigorta ile karşılaştırma yapabilir mi?", "Evet, rekabete dayalı karşılaştırmalı teklifleri inceleme olanağına sahip olurlar."),
        ("E-sigortanın mekânsal faydası nedir?", "Müşterilerin bulundukları herhangi bir yerden sigorta işlemlerinden yararlanabilmesidir."),

        # --- BÖLUM 4: AVANTAJLAR VE FAYDALAR (ŞİRKET VE SEKTÖR) ---
        ("E-sigortanın sigorta şirketleri için en büyük etkisi nedir?", "En büyük etki, üretim ve pazarlama gibi genel giderlerde maliyetlerin düşmesidir."),
        ("E-sigorta, aracı komisyonlarını nasıl etkiler?", "Sigorta şirketlerinin poliçeyi doğrudan satması, aracılara ödenen komisyonu azaltmaktadır."),
        ("E-sigortacılık şirketlerin küreselleşmesini sağlar mı?", "Evet, zaman ve mekân kısıtlamalarını ortadan kaldırdığı için sigorta şirketlerinin küreselleşmesine olanak tanır."),
        ("Şirketler için e-sigortacılığın rekabet avantajı nedir?", "Pazara yeni girecek şirketlerin sigorta pazarına kolay erişmelerine olanak verir."),
        ("E-sigorta, veri yönetimi açısından ne gibi bir fayda sağlar?", "Müşterilerden toplanan veriler depolanarak veri madenciliği uygulamalarında kullanılabilmektedir."),
        ("Geleneksel sigortacılıkta veri depolama nasıldır?", "Geleneksel sigortacılıkta veri depolama daha zordur."),
        ("E-sigortacılık yeni ürün geliştirmeyi nasıl etkiler?", "Müşterilerin özel ihtiyaçlarına göre ürünler oluşturulabildiği için yeni ürün geliştirmede avantaj yaratmaktadır."),
        ("E-sigortanın aracılar için faydası nedir?", "Hızlı iletişim kolaylık yaratmakta, potansiyel müşteri profili çıkarılabilmektedir."),
        ("Şirketler e-sigorta ile satış hacmini artırabilir mi?", "Evet, e-sigortacılık satış hacminin artmasına yardımcı olur."),

        # --- BÖLUM 5: ZORLUKLAR, SORUNLAR VE ELEŞTİRİLER ---
        ("E-sigortacılığın başlangıcındaki en büyük tartışma konusu neydi?", "En büyük sorunların başında 'güven' gelmektedir."),
        ("Müşteriler neden e-sigortaya karşı güvensizlik duymuştur?", "Özel ve gizli bilgilerinin internet ortamında başkasının eline geçme tehlikesi müşterilerde güvensizlik yaratmaktadır."),
        ("Bu güvensizliğin temel nedeni ne olarak düşünülmüştür?", "Endüstride güvenilir bilgi alışverişine izin veren uluslararası standartların eksikliği olarak düşünülmüştür."),
        ("Sigorta sektörü e-ticareti benimsemede hızlı mıydı?", "Hayır, birçok çalışmada sigorta sektörünün e-ticareti benimsemede geç kaldığı ifade edilmiştir."),
        ("Şirketlerin e-sigortacılığı benimsemede yavaş kalmasının bir sebebi nedir?", "E-sigortacılıkla ilgili altyapıyı oluşturmadaki tecrübesizlikleri bir sebep olarak gösterilmiştir."),
        ("Başlangıçta hangi sigorta türleri e-sigortaya daha az uygun görülüyordu?", "Hayat ve sağlık poliçeleri gibi ürünler, kasko ve konut gibi yalın sigortalara göre daha az uygun bulunuyordu."),
        ("Komplike sigorta poliçelerinin internetten satışı için ne gereklidir?", "Şirketlerin belirli standartları sağlamaları gerekmektedir."),
        ("Bazı ülkelerde e-sigorta işlemlerini kısıtlayan nedir?", "Düzenleyici, denetimsel ve teknolojik kısıtlamalar elektronik işlemlerin yapılmasına izin vermeyebilir."),
        ("E-sigortacılıkla ilgili akademik çalışmalar yeterli mi?", "Hayır, metinde akademik çalışmaların da yeterince olmadığı vurgulanmaktadır."),

        # --- BÖLUM 6: DEVLETİN ROLÜ, REKABET VE PİYASA ETKİLERİ ---
        ("E-sigortacılık konusunda devletin sorumlulukları nelerdir?", "Gerekli teknik ve hukuki altyapıyı oluşturmak, uygulamaları denetlemek ve uluslararası standartlara uygun düzenlemeler yapmaktır."),
        ("Devlet, e-sigortacılık sorunlarını çözmek için ne yapmalıdır?", "Kamu, özel sektör, sivil toplum örgütleri ve üniversiteler arasında işbirliği sağlamalıdır."),
        ("E-sigorta piyasadaki rekabeti nasıl etkiler?", "Pazara giriş maliyetlerini düşürdüğü ve yeni şirketlerin piyasaya girmesini kolaylaştırdığı için rekabeti artırır."),
        ("Ürün bilgilerinin şeffaf olması rekabeti neden artırır?", "Çünkü ürün fiyat ve özelliklerinin internetten rahatlıkla karşılaştırılabilmesini sağlar."),
        ("E-sigortacılığın temel araçları nelerdir?", "E-sigortanın temel araçları bilgi ve iletişimdir."),
        ("Pazarın genişlemesi için e-sigortaya geçiş önemli midir?", "Evet, e-sigortaya geçiş, pazarın genişlemesini sağlayacak ve rekabeti düzeltecektir."),
        ("E-sigortanın rekabete etkisindeki en büyük etken nedir?", "Zaman ve mekânda kısıt olmamasının büyük bir etken olduğu görülebilir."),

        # --- BÖLUM 7: ÖDEME YÖNTEMLERİ VE İŞLEYİŞ ---
        ("E-sigortacılıkta kullanılan ödeme araçları e-ticaretten farklı mıdır?", "Evet, e-sigortacılıkta kullanılan ödeme araçlarının e-ticaretten farklı olduğu görülmüştür."),
        ("E-sigortadaki ödeme türleri nelerdir?", "Ödemeler, Nakit Ödemeler ve Nakit Dışı Ödemeler olarak iki grupta incelenmektedir."),
        ("E-sigortada nakit dışı ödeme yolları nelerdir?", "Çekler, senetler, kredi kartı gibi kart tabanlı araçlar, sanal kredi kartı gibi yazılım tabanlı araçlar, çevrimiçi havale ve EFT'dir."),
        ("Kredi kartı, e-sigortada nasıl bir ödeme aracıdır?", "Kart tabanlı bir nakit dışı ödeme aracıdır."),
        ("Sanal kredi kartı hangi ödeme grubuna girer?", "Yazılım tabanlı nakit dışı ödeme araçları grubuna girer."),
        ("Acenteler e-sigortacılık yaparken nakit ödeme alabilir mi?", "Evet, acentelerinden çek, senet ve nakit ödeme kabul edebilmektedirler."),

        # --- BÖLUM 8: KARŞILAŞTIRMALAR VE FARKLILIKLAR ---
        ("Geleneksel sigortacılık ile e-sigortacılık arasındaki en temel ürün farkı nedir?", "Geleneksel sigortacılıkta hazır ürünler seçilirken, e-sigortacılıkta müşteriye özel ürünler oluşturulabilmektedir."),
        ("Çevrimiçi bankacılık ile e-sigortanın gelişim hızı nasıl karşılaştırılır?", "İnternetin sigorta endüstrisindeki gelişimi, çevrimiçi bankacılık ile karşılaştırıldığında biraz daha temkinli ve yavaş olmuştur."),
        ("İlk dönem web siteleri ile günümüz e-sigorta platformları arasındaki fark nedir?", "Başlangıçta web siteleri sadece şirket ve ürünler hakkında bilgi verme amacı güderken, günümüzde tüm işlemler online yapılabilmektedir."),

        # --- BÖLUM 9: ÇEŞİTLİ VE DETAY SORULAR ---
        ("Yazıcı ve Yanık (2002) e-sigortacılık hakkında ne belirtmiştir?", "Müşterilerin internet üzerinden poliçe satın alabildiğini, sözleşme düzenleyebildiğini ve ödeme yapabildiğini belirtmişlerdir."),
        ("Sekolovska (2012) şirket yöneticilerinin endişesi hakkında ne demiştir?", "Yöneticilerin, fiili uygulama sırasında ortaya çıkacak sorunların e-sigortadan beklenen faydaları azaltacağını düşündüğünü vurgulamıştır."),
        ("McCarthy ve ark. (2000) sigorta endüstrisindeki en önemli zorluk olarak neyi belirtmiştir?", "Uluslararası standartların eksikliğini en önemli zorluklardan biri olarak belirtmişlerdir."),
        ("Bu çalışmanın ilk amacı nedir?", "Türkiye’de e-sigortacılıkta yaşanan değişimi ortaya koymaktır."),
        ("Bu çalışmanın ikinci amacı nedir?", "Hayat sigortalarında e-sigorta prim üretiminin gelecek dönem öngörüsünü elde etmektir."),
        ("Çalışmada hangi veri kaynağı kullanılmıştır?", "Türkiye Sigortalar Birliğinin yayınladığı e-sigorta prim üretim tabloları incelenmiştir."),
        ("Analiz yöntemi olarak neden doğrusal olmayan otoregresif sinir ağları tercih edilmiştir?", "Hem doğrusal hem de doğrusal olmayan serilerin analizinde kullanılabilmesi, ön koşul gerektirmemesi ve esnek yapısından dolayı tercih edilmiştir."),
        ("E-sigortanın hacimlerini artırmak için ne gereklidir?", "Sigorta şirketlerinin hizmetlerini geliştirmesi ve müşterilerin güvenini sağlayacak ortamları oluşturması gerekmektedir."),
        ("E-sigortanın gelişiminde neyin önemi vardır?", "Tüm pazarlama hizmetlerinin internet ortamında yer alması önem taşımaktadır."),
        ("Tüketicilerin güvenini kazanmak için ürünler nasıl olmalıdır?", "Ürünleri basitleştirmek, daha şeffaf hale getirmek ve gereksiz maliyetleri azaltmak gerekir."),

         # --- BÖLÜM 1: GENEL SİGORTA KAVRAMLARI ---
        ("Sigorta primi ne demek?", "Sigorta primi, sigortanın sağladığı güvenceden yararlanmak için sigorta şirketine ödediğiniz ücrettir. Aylık veya yıllık olarak ödenebilir."),
        ("Poliçe nedir?", "Poliçe, sigorta yaptıran kişi ile sigorta şirketi arasındaki sözleşmedir. Sigortanın şartlarını, neleri kapsadığını ve ne kadar teminat verdiğini gösteren belgedir."),
        ("Muafiyet ne anlama geliyor?", "Muafiyet, bir hasar durumunda sigortalının karşılayacağı, önceden belirlenmiş küçük bir hasar tutarıdır. Hasarın bu kısmı size, üstü sigortaya aittir."),
        ("Hasarsızlık indirimi ne işe yarar?", "Hasarsızlık indirimi, sigorta süresi boyunca herhangi bir hasar talebinde bulunmadığınızda, bir sonraki yıl poliçenizi yenilerken size uygulanan fiyattaki indirimdir."),
        ("Zeyilname nedir, neden düzenlenir?", "Zeyilname, mevcut sigorta poliçeniz üzerinde adres, plaka değişikliği gibi güncellemeler yapmak için düzenlenen ek belgedir."),
        ("Sigorta bedeli ne demektir?", "Sigorta bedeli, bir hasar anında sigorta şirketinin size ödeyeceği en yüksek tazminat tutarıdır. Poliçenizde bu miktar belirtilir."),
        ("Acente ile broker arasında ne fark var?", "Acente, tek bir sigorta şirketinin ürünlerini satar. Broker ise birden fazla sigorta şirketinden sizin adınıza teklif alıp karşılaştırma yaparak size en uygununu sunar."),
        ("Sigortamı istediğim zaman iptal edebilir miyim?", "Evet, genellikle sigortanızı istediğiniz zaman iptal edebilirsiniz ancak şirketin belirlediği koşullara göre prim iadesi alıp alamayacağınız değişebilir."),
        ("Neden farklı şirketlerden teklif almalıyım?", "Çünkü her şirketin risk değerlendirmesi ve fiyatlandırması farklıdır. Teklif alarak aynı teminatları daha uygun fiyata bulma şansınız artar."),
        ("Hasar durumunda ilk kimi aramalıyım?", "Hasar anında ilk olarak sigorta şirketinizin hasar ihbar hattını veya acentenizi aramalısınız."),

        # --- BÖLÜM 2: ARAÇ SİGORTALARI (TRAFİK VE KASKO) ---
        ("Zorunlu trafik sigortası neyi kapsar?", "Zorunlu trafik sigortası, bir kaza anında sizin kusurunuzdan dolayı karşı tarafın aracına veya kişilere verdiğiniz zararları karşılar. Sizin kendi aracınızın hasarını karşılamaz."),
        ("Kasko sigortası ne işe yarar?", "Kasko, kendi aracınızın başına gelebilecek çarpma, yanma, çalınma gibi zararları karşılayan sigortadır. Zorunlu değildir."),
        ("Trafik sigortası ile kasko arasındaki en temel fark nedir?", "En temel fark şudur: Trafik sigortası karşı tarafın zararını, kasko ise sizin kendi aracınızın zararını karşılar."),
        ("Kaza yaptım, trafik sigortası benim masrafımı öder mi?", "Hayır, zorunlu trafik sigortası sadece karşı tarafa verdiğiniz hasarı öder. Kendi masraflarınız için kasko sigortanızın olması gerekir."),
        ("Kasko yaptırmak zorunlu mu?", "Hayır, kasko sigortası yaptırmak tamamen isteğe bağlıdır. Sadece zorunlu trafik sigortası yasal bir zorunluluktur."),
        ("Arabamın kasko değeri nasıl belirleniyor?", "Türkiye Sigorta Birliği (TSB) tarafından her ay yayınlanan araç marka, model ve yaşına göre belirlenen bir liste üzerinden hesaplanır."),
        ("Küçük bir kaza yaptım, hasarsızlık indirimim bozulur mu?", "Poliçenizde 'mini onarım' teminatı varsa, bu teminat kapsamındaki küçük hasarlar (çizik, göçük vb.) hasarsızlık indiriminizi bozmadan tamir edilebilir."),
        ("İkame araç ne demek?", "Kaza sonrası aracınızın serviste onarımda olduğu süre boyunca, sigorta şirketinin size geçici olarak kullanmanız için verdiği yedek araçtır."),
        ("Kasko fiyatımı ne düşürür?", "Aracınızın yaşı, modeliniz, hasar geçmişiniz (hasarsızlık indirimi), mesleğiniz ve yaşadığınız şehir gibi faktörler kasko fiyatınızı etkiler."),

        # --- BÖLÜM 3: SAĞLIK SİGORTALARI (ÖSS VE TSS) ---
        ("Özel sağlık sigortası (ÖSS) nedir?", "Anlaşmalı özel hastanelerde muayene, tahlil, ameliyat gibi sağlık hizmetlerinden daha uygun şartlarda faydalanmanızı sağlayan sigorta türüdür."),
        ("Tamamlayıcı sağlık sigortası (TSS) ne işe yarar?", "SGK'lı iseniz, SGK ile anlaşmalı özel hastanelere gittiğinizde ödemeniz gereken fark ücretini sizin yerinize ödeyen sigortadır."),
        ("ÖSS ile TSS arasındaki fark nedir?", "Özel Sağlık Sigortası (ÖSS) daha geniş bir hastane ağı sunar ve SGK'lı olma şartı aramaz. Tamamlayıcı Sağlık Sigortası (TSS) ise sadece SGK anlaşmalı kurumlarda geçerlidir ve daha ekonomiktir."),
        ("Sağlık sigortası her hastanede geçerli mi?", "Hayır, sadece sigorta şirketinizin anlaşmalı olduğu hastane ve sağlık kurumlarında geçerlidir. Poliçenizi alırken anlaşmalı kurumlar listesini kontrol etmelisiniz."),
        ("Yatarak ve ayakta tedavi ne demek?", "Ayakta tedavi; doktor muayenesi, röntgen, tahlil gibi hastanede yatış gerektirmeyen durumları kapsar. Yatarak tedavi ise ameliyat gibi en az 24 saat hastanede kalmayı gerektiren durumları kapsar."),
        ("Geçmişten gelen bir hastalığım var, sigorta bunu karşılar mı?", "Genellikle sigorta poliçeleri, sözleşme başlangıcından önce var olan (mevcut) hastalıkları kapsamaz. Bu durum şirkete göre değişebilir."),
        ("Sağlık sigortası doğum masraflarını karşılar mı?", "Poliçenize 'doğum teminatı' ekletirseniz, hamilelik süresince kontrolleri ve doğum masraflarını karşılar. Genellikle bu teminat için bir bekleme süresi uygulanır."),

        # --- BÖLÜM 4: KONUT SİGORTALARI (DASK VE KONUT) ---
        ("DASK nedir ve zorunlu mudur?", "DASK (Doğal Afet Sigortaları Kurumu), deprem ve depremden kaynaklanan yangın, tsunami gibi risklere karşı binanızı güvence altına alan zorunlu bir sigortadır."),
        ("DASK sadece depremi mi kapsar?", "Evet, DASK'ın ana teminatı depremdir. Depremin doğrudan neden olduğu maddi zararları ve deprem sonucu meydana gelen yangın, infilak, tsunami ve yer kayması hasarlarını karşılar."),
        ("Konut sigortası ne işe yarar?", "Konut sigortası, evinizi ve isteğe bağlı olarak eşyalarınızı yangın, hırsızlık, sel, su baskını gibi DASK'ın kapsamadığı birçok riske karşı korur."),
        ("DASK ile konut sigortası aynı şey mi?", "Hayır, aynı şey değildir. DASK zorunludur ve sadece deprem kaynaklı bina hasarlarını karşılar. Konut sigortası isteğe bağlıdır ve hırsızlıktan su baskınına kadar çok daha geniş riskleri kapsar."),
        ("Konut sigortası yaptırırsam eşyalarım da güvende mi?", "Evet, poliçenize 'eşya teminatı' eklerseniz, evinizdeki eşyalar da belirtilen risklere karşı güvence altına alınır."),
        ("Kiracıyım, konut sigortası yaptırabilir miyim?", "Evet, kiracılar da sadece kendi eşyalarını güvence altına almak için konut sigortası yaptırabilirler."),
        ("Evime su bastı, DASK karşılar mı?", "Hayır, DASK karşılamaz. Su baskını gibi hasarlar için konut sigortanızın olması gerekmektedir."),

        # --- BÖLÜM 5: DİĞER SİGORTALAR VE SÜREÇLER ---
        ("Hayat sigortası neden yaptırmalıyım?", "Hayat sigortası, vefat veya kalıcı sakatlık gibi durumlarda sevdiklerinize veya size finansal bir güvence sağlamak için yapılır."),

        ("Seyahat sağlık sigortası nedir?", "Yurt içi veya yurt dışı seyahatleriniz sırasında başınıza gelebilecek ani hastalık veya kaza gibi acil sağlık sorunlarına karşı sizi güvence altına alan bir sigortadır."),
        ("Vize için seyahat sigortası zorunlu mu?", "Evet, özellikle Schengen ülkeleri başta olmak üzere birçok ülke vize başvurusu sırasında seyahat sağlık sigortasını zorunlu tutmaktadır."),
        ("Seyahat sigortası valizim kaybolursa ödeme yapar mı?", "Poliçenizde 'bagaj kaybı teminatı' varsa, seyahat sırasında valizinizin kaybolması veya çalınması durumunda sigorta şirketi zararınızı karşılar."),
        ("Hasar ihbarı nasıl yapılır?", "Hasar durumunda, sigorta şirketinizin çağrı merkezini arayarak, mobil uygulamasından veya web sitesinden online olarak hasar ihbarında bulunabilirsiniz."),
        ("Sigorta şirketi hasarımı ne kadar zamanda öder?", "Gerekli belgeleri eksiksiz teslim ettikten sonra, dosyanın incelenme ve eksper raporu sürecine bağlı olarak ödeme süresi değişir, ancak genellikle yasal süreçler dahilinde kısa sürede sonuçlanır."),
    ]

    # Dataset formatları
    formatted_data = []
    for question, answer in qa_data:
        # Çeşitli formatlar ekle
        formatted_data.extend([
            f"Soru: {question}\nCevap: {answer}",
            f"Q: {question}\nA: {answer}",
            f"Kullanıcı: {question}\nAsistan: {answer}",
            f"İnsan: {question}\nYapay Zeka: {answer}",
            f"{question}\n{answer}",
        ])

    # Sizin metinlerinize uygun, konuya özel konuşma formatları
    conversations = [
        "İnsan: Merhaba, e-sigortacılık hakkında bilgi alabilir miyim?\nAsistan: Merhaba! Tabii ki. E-sigortacılık, sigorta işlemlerini internet üzerinden online olarak yapmanızı sağlayan bir sistemdir.\nİnsan: Peki bunun bana ne gibi bir faydası olur?\nAsistan: En büyük faydası, farklı şirketlerin tekliflerini kolayca karşılaştırıp size en uygun poliçeyi zaman kaybetmeden, daha uygun fiyata bulabilmenizdir.",
        "Kullanıcı: İnternetten sigorta yaptırmak güvenli mi? Kişisel bilgilerim konusunda endişelerim var.\nAsistan: Bu çok yerinde bir soru. E-sigortacılığın ilk yıllarında güven en büyük sorunlardan biriydi. Ancak günümüzde gelişen teknoloji ve yasal düzenlemelerle birlikte güvenli bir altyapı oluşturulmuştur. Yine de her zaman güvenilir ve bilinen şirketlerle çalışmanız önemlidir.\nKullanıcı: Anladım, teşekkür ederim.\nAsistan: Rica ederim! Başka sorularınız olursa çekinmeden sorabilirsiniz.",
        "Soru: B2B ve B2C e-sigorta arasındaki fark nedir?\nCevap: B2B, yani şirketten şirkete modelde, sigorta şirketleri acenteler gibi aracılarla online çalışır. B2C modelinde ise sigorta şirketleri veya aracıları, doğrudan son tüketiciye yani müşteriye internet üzerinden satış yapar."
    ]

    # Konuşmaları çoğalt
    formatted_data.extend(conversations * 30)

    # Ana veriyi çoğaltarak dataset boyutunu artır
    final_data = formatted_data * 10

    # Hugging Face Dataset objesi olarak döndür
    return Dataset.from_dict({"text": final_data})

# --- ANA KOD AKIŞI ---

# Dataset'i yükle (manuel fonksiyonumuzu çağıracak)
dataset = load_turkish_dataset()
print(f"📊 Dataset yüklendi: {len(dataset)} örnek")

# Dataset temizleme fonksiyonu
def clean_dataset(example):
    """Dataset'teki metinleri temizler"""
    text = example["text"].strip()

    # Çok kısa veya çok uzun metinleri filtrele
    if len(text) < 20 or len(text) > 1024:
        return {"text": ""}

    # Boş veya anlamsız metinleri filtrele
    if not text or text.count('\n') > 10:
        return {"text": ""}

    return {"text": text}

# Dataset'i temizle ve boş satırları filtrele
dataset = dataset.map(clean_dataset, num_proc=4) # İşlemi hızlandırmak için çoklu işlemci kullan
dataset = dataset.filter(lambda x: len(x["text"]) > 20)

# Veri setini Eğitim (Train) ve Değerlendirme (Test) olarak ayır
dataset = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

print(f"✅ Dataset hazır: {len(train_dataset)} train, {len(eval_dataset)} test")
print(f"📝 Örnek Veri: {train_dataset[0]['text'][:150]}...")

# 5. TOKENIZATION

In [ ]:
print("\n🔤 Tokenization başlıyor...")

def tokenize_function(examples):
    """Gelişmiş tokenization fonksiyonu"""

    # Tokenize et
    result = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",  # Padding'i burada uygula
        max_length=256,        # Belirlediğimiz max length
        return_tensors=None    # List olarak döndür
    )

    # Labels'i input_ids'in kopyası yap
    result["labels"] = [input_ids.copy() for input_ids in result["input_ids"]]

    return result

# Tokenization işlemi
print("🔄 Training dataset tokenization...")
tokenized_train = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=train_dataset.column_names,
    num_proc=4 if os.cpu_count() > 4 else 1
)

print("🔄 Evaluation dataset tokenization...")
tokenized_eval = eval_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=eval_dataset.column_names,
    num_proc=4 if os.cpu_count() > 4 else 1
)

print("✅ Tokenization tamamlandı!")
print(f"📊 Tokenized train: {len(tokenized_train)}")
print(f"📊 Tokenized eval: {len(tokenized_eval)}")

# 6. BASIT TRAİNİNG ARGS

In [ ]:
print("\n⚙️ Training arguments ayarlanıyor...")

training_args = TrainingArguments(
    # Çıktı dizini
    output_dir="./turkish-model-final",
    overwrite_output_dir=True,

    # Training parametreleri
    num_train_epochs=3,                    # 3 epoch yeterli
    per_device_train_batch_size=4,         # Batch size
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,         # Effective batch size = 4*2 = 8

    # Öğrenme hızı
    learning_rate=5e-5,                    # Orta seviye learning rate
    weight_decay=0.01,                     # Regularization
    warmup_steps=100,                      # Warmup

    # Evaluation
    eval_strategy="steps",                 # Step bazında değerlendirme
    eval_steps=100,                        # Her 100 step'te değerlendirme
    logging_steps=50,                      # Logging sıklığı

    # Save stratejisi
    save_strategy="steps",
    save_steps=200,                        # Her 200 step'te kaydet
    save_total_limit=2,                    # En fazla 2 checkpoint tut

    # Performans optimizasyonu
    dataloader_num_workers=2,              # Paralel veri yükleme
    fp16=torch.cuda.is_available(),        # Mixed precision (GPU varsa)

    # En iyi modeli yükle
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    # Raporlama kapalı
    report_to=[],

    # Memory optimizasyonu
    remove_unused_columns=False,
    dataloader_pin_memory=False,
)

print("✅ Training arguments hazır!")


# 7. DATA COLLATOR

In [ ]:
print("\n📦 Data collator ayarlanıyor...")

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # Causal LM (GPT-style)
    # pad_to_multiple_of=8,  # Padding artık tokenization'da yapılıyor
)

print("✅ Data collator hazır!")

# 8. TRAINER SETUP

In [ ]:
print("\n🏃‍♂️ Trainer setup...")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],  # Early stopping
)

print("✅ Trainer hazır!")
print(f"📊 Total training steps: {len(tokenized_train) // training_args.per_device_train_batch_size * training_args.num_train_epochs}")


# 9. TRAINING BAŞLAT

In [ ]:

print("\n🚀 TRAINING BAŞLIYOR!")
print("=" * 50)
print(f"Model: {MODEL_NAME}")
print(f"Dataset size: {len(tokenized_train)} training, {len(tokenized_eval)} eval")
print(f"Epochs: {training_args.num_train_epochs}")
print(f"Batch size: {training_args.per_device_train_batch_size}")
print(f"Learning rate: {training_args.learning_rate}")
print("=" * 50)

# Tahmini süre hesaplama
estimated_time = len(tokenized_train) * training_args.num_train_epochs / (training_args.per_device_train_batch_size * 60)
print(f"⏱️ Tahmini süre: {estimated_time:.1f} dakika")
print()

# Training başlat
trainer.train()

print("\n🎉 TRAINING TAMAMLANDI!")


# 10. MODELİ KAYDETME

In [ ]:
print("\n💾 Model kaydediliyor...")

# Model ve tokenizer'ı kaydet
trainer.save_model("./turkish-model-final")
tokenizer.save_pretrained("./turkish-model-final")

print("✅ Model başarıyla kaydedildi: ./turkish-model-final")

# Training geçmişini kaydet
if trainer.state.log_history:
    print(f"📊 Training loss: {trainer.state.log_history[-1].get('train_loss', 'N/A')}")
    print(f"📊 Eval loss: {trainer.state.log_history[-1].get('eval_loss', 'N/A')}")


# 11. DÜZGÜN TEST FONKSİYONU

In [ ]:
print("\n🧪 Gelişmiş test fonksiyonu hazırlanıyor...")

def clean_response(text):
    """Cevapları temizle ve tekrarları kaldır"""
    import re

    # Boş satırları temizle
    text = re.sub(r'\n\s*\n', '\n', text)

    # Aynı cümlenin tekrarını engelle
    sentences = text.split('.')
    cleaned_sentences = []
    seen = set()

    for sentence in sentences:
        sentence = sentence.strip()
        if sentence and sentence not in seen and len(sentence) > 5:
            cleaned_sentences.append(sentence)
            seen.add(sentence)
        if len(cleaned_sentences) >= 3:  # En fazla 3 cümle
            break

    result = '. '.join(cleaned_sentences)
    if result and not result.endswith('.'):
        result += '.'

    return result

def test_model_fixed(prompt, max_new_tokens=80, temperature=0.7, top_p=0.9):
    """
    TEKRARLARİ ÖNLEYEN gelişmiş test fonksiyonu

    Args:
        prompt (str): Giriş metni
        max_new_tokens (int): Maksimum YENİ token sayısı
        temperature (float): Yaratıcılık (düşük = daha tutarlı)
        top_p (float): Nucleus sampling
    """

    # Input'u tokenize et
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=100)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # GÜÇLÜ TEKRAR ÖNLEMELİ GENERATION
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,  # Max LENGTH yerine MAX NEW TOKENS
            temperature=temperature,
            do_sample=True,
            top_p=top_p,
            top_k=40,                       # Daha kısıtlı seçim
            repetition_penalty=1.3,         # YÜKSEK tekrar cezası
            no_repeat_ngram_size=2,         # 2-gram tekrarı engelle
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            num_return_sequences=1,
            # early_stopping parametresi kaldırıldı
        )

    # Sadece yeni üretilen kısmı al
    new_tokens = outputs[0][inputs['input_ids'].shape[1]:]
    response = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()

    # Cevabı temizle
    response = clean_response(response)

    # Çok kısa cevapları genişlet
    if len(response) < 10:
        response = "Size yardımcı olmaya çalışıyorum. Başka soru var mı?"

    # Çok uzun cevapları kısalt
    if len(response) > 200:
        sentences = response.split('.')[:2]  # İlk 2 cümle
        response = '. '.join(sentences) + '.'

    return {
        "prompt": prompt,
        "response": response,
        "length": len(response)
    }

def smart_test(prompt, **kwargs):
    """Akıllı test - farklı parametrelerle dene"""
    # Önce düşük temperature ile dene
    result = test_model_fixed(prompt, temperature=0.6, **kwargs)

    # Eğer çok kısa ise temperature artır
    if len(result["response"]) < 20:
        result = test_model_fixed(prompt, temperature=0.8, **kwargs)

    return result

print("✅ Test fonksiyonları hazır - TEKRAR PROBLEMİ ÇÖZÜLDÜ!")

def batch_test_fixed(prompts, **kwargs):
    """Gelişmiş batch test"""
    results = []
    for i, prompt in enumerate(prompts, 1):
        print(f"🧪 Test {i}/{len(prompts)}: {prompt[:30].strip()}...")
        result = smart_test(prompt, **kwargs)
        results.append(result)
    return results

# 12. TEST ÖRNEKLERI

In [ ]:
print("\n🧪 MODEL TEST EDİLİYOR!")
print("=" * 60)

# Test prompt'ları - BASIT VE NET
test_prompts = [
    "Soru: Merhaba\nCevap:",
    "Soru: Python nedir\nCevap:",
    "Soru: Yapay zeka nedir\nCevap:",
    "Soru: JavaScript nedir\nCevap:",
    "Soru: Türkiye başkenti\nCevap:",
    "Kullanıcı: Teşekkürler\nAsistan:",
    "Soru: İyi günler\nCevap:",
    "Kullanıcı: HTML nedir\nAsistan:",
]

print("🚀 GELİŞTİRİLMİŞ TEST BAŞLIYOR!\n")

# Her prompt için test
for i, prompt in enumerate(test_prompts, 1):
    print(f"{'='*60}")
    print(f"🧪 TEST {i}/8")
    print(f"Prompt: {prompt.strip()}")

    # Test et
    result = smart_test(prompt, max_new_tokens=50)

    print(f"✅ Response: {result['response']}")
    print(f"📏 Uzunluk: {result['length']} karakter")
    print()

# EXTRA: Interaktif test
print(f"{'='*60}")
print("🎯 İNTERAKTİF TEST - Kendi prompt'unuzu test edin:")

def interactive_test():
    """Kullanıcıdan prompt alıp test et"""
    sample_prompts = [
        "Soru: Programlama nasıl öğrenilir\nCevap:",
        "Kullanıcı: Veri bilimi nedir\nAsistan:",
        "Soru: Web tasarımı için ne önerirsiniz\nCevap:"
        "Soru: Gün kavramını bana anlat\nCevap:"
    ]

    print("\n💡 Örnek prompt'lar:")
    for i, p in enumerate(sample_prompts, 1):
        print(f"{i}. {p}")

    print("\n🔧 Test fonksiyonu hazır!")
    print("Kullanım: result = smart_test('Soru: ... \\nCevap:')")

interactive_test()

print(f"\n{'='*60}")
print("🎉 TÜM TESTLER TAMAMLANDI!")
print(f"✅ Model başarıyla eğitildi ve test edildi!")
print(f"📁 Model dizini: ./turkish-model-final")
print(f"🔧 Toplam parametre: ~{model.num_parameters()/1e6:.1f}M")
print(f"📚 Training veri: {len(tokenized_train)} örnek")
print(f"🎯 Model: {MODEL_NAME}")

# Kullanım talimatları
print(f"\n💡 MODEL KULLANIM TALİMATLARI:")
print("1. Model './turkish-model-final' dizininde kaydedildi")
print("2. test_model() fonksiyonu ile test edebilirsiniz")
print("3. Farklı temperature değerleri ile yaratıcılığı ayarlayabilirsiniz")
print("4. Model Türkçe soru-cevap formatında eğitilmiştir")

print(f"\n🔄 Yeniden yüklemek için:")
print("loaded_model = AutoModelForCausalLM.from_pretrained('./turkish-model-final')")
print("loaded_tokenizer = AutoTokenizer.from_pretrained('./turkish-model-final')")

In [ ]:
def run_interactive_model_context():
    print("🎯 Model ile sohbet başlıyor! Çıkmak için 'çıkış' yazın.\n")

    # Bağlamı saklamak için liste
    conversation_history = []

    while True:
        user_input = input("Kullanıcı: ")
        if user_input.lower() == "çıkış":
            print("💤 Sohbet sonlandırıldı.")
            break

        # Prompt: önceki konuları ve yeni soruyu birleştir
            # Konuşmayı bağlam listesine ekle
        conversation_history.append((user_input, response))

        # Talimat: Sadece tek cümle, doğru ve bilgi odaklı
        context_text = ""
        for q, a in conversation_history[-5:]:  # son 5 soruyu hatırlayacak
            context_text += f"Soru: {q}\nCevap: {a}\n"

        prompt = (
    f"Sadece tek cümle ile, doğru ve açık bilgi ver.\n"
    f"Hiçbir selamlaşma, yorum veya soru sorma ekleme.\n"
    f"Soru: {user_input}\n"
    f"Cevap:"
)


        # Modeli çalıştır
        result = smart_test(prompt, max_new_tokens=60)

        # Tek satır yanıt
        response = result['response'].split("\n")[0].strip()
        print(f"Asistan: {response}\n")



# Fonksiyonu başlat
run_interactive_model_context()
